In [1]:
import cv2 as cv
import numpy as np
import matplotlib.pyplot as plt
from keras.datasets import mnist

2023-11-19 21:37:44.887888: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE4.1 SSE4.2 AVX AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


Загрузка "обучающего" датасета MNIST. Получение векторов признаков

In [2]:
(train_X, train_Y), (test_X, test_Y) = mnist.load_data()

In [3]:
train_X.shape

(60000, 28, 28)

In [4]:
test_X.shape

(10000, 28, 28)

In [5]:
train_ftrs = [0.] * 60000
test_ftrs=[0.] * 10000

In [6]:
for i in range(60000):
    train_X[i] = cv.threshold(train_X[i], 0, 255, cv.THRESH_BINARY+cv.THRESH_OTSU)[1]
    train_X[i] //= 255
    train_ftrs[i] = np.ndarray.flatten(train_X[i])
    if i < 10000:
        test_X[i] = cv.threshold(test_X[i], 0, 255, cv.THRESH_BINARY+cv.THRESH_OTSU)[1]
        test_X[i] //= 255
        test_ftrs[i] = np.ndarray.flatten(test_X[i])

Разные метрики для подсчета расстояний между признаками

In [22]:
def euclid(a, b):
    if len(a) != len(b):
        print('Inconsistent vectors')
        return -1
    x = a.astype(int)
    y = b.astype(int)
    dists = x - y
    dists **= 2
    dist = dists.sum()
    return dist ** 0.5

Методы для извлечения признаков

In [48]:
def feature_extract(img):
    pass

Классификатор

In [8]:
def knn(input_ftr, train_ftr, train_lbl, k=1, weighted=False):
    distances = []
    
    # calculate distances to all training set examples and store them with labels
    # [(lbl, dist)]
    for i in range(len(train_ftr)):
        distances.append((train_lbl[i], euclid(input_ftr, train_ftr[i])))

    # sort by distance and choose k nearest
    distances.sort(key=lambda x: x[1])
    distances = distances[:k]

    w = [1.] * k
    # there should be weighting process for chosen k neighbours
    if weighted:
        pass

    # judgment = {lbl: cumulative weight}
    judgment = dict()
    for i in range(len(distances)):
        lbl = distances[i][0]
        if lbl not in judgment:
            judgment[lbl] = w[i] 
        else:
            judgment[lbl] += w[i]

    # sort labels by prevalence and choose the answer
    judgment = list(judgment.items())
    judgment.sort(key=lambda x: x[1], reverse=True)
    return judgment[0][0]

In [27]:
def hit_rate(pred, test):
    tries = len(pred)
    hits = 0
    for i in range(tries):
        if pred[i] == test[i]:
            hits+=1
    return hits/tries

In [31]:
answers = [None]*100
for i in range(100):
    answers[i] = knn(test_ftrs[i], train_ftrs, train_Y)

In [32]:
hit_rate(answers, test_Y)

0.97